# Reverse Engineering Transforms Analysis

This notebook explores the advanced signal transforms proposed in the Anomaly Reverse Engineering Report:

1. **Nonlinear Power Transforms (x², x⁴)** - Unmask phase structure of digital modulations
2. **Symbol-Lagged Correlation (τ ≈ 3.4)** - Highlight phase relationships between symbols
3. **Instantaneous Frequency Analysis (dφ/dt)** - Reveal frequency shift patterns
4. **Cyclostationary Feature Extraction (SCF)** - Exploit periodic statistics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import kurtosis, skew
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

from data_utils import load_train_data, load_test_anomalies, filter_by_snr, create_binary_labels

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

In [ ]:
# Load data
print("Loading data...")
train_signals, train_labels, train_snr = load_train_data()
test_signals, test_labels, test_snr = load_test_anomalies()

# Filter SNR=0 from training (not present in test)
train_signals, train_labels, train_snr = filter_by_snr(
    train_signals, train_labels, train_snr, [10, 20, 30]
)

# Binary labels for test
test_binary = create_binary_labels(test_labels)

print(f"Train: {len(train_signals)} samples, Classes: {np.unique(train_labels)}")
print(f"Test: {len(test_signals)} samples, Classes: {np.unique(test_labels)}")
print(f"Test anomalies: {test_binary.sum()} ({100*test_binary.mean():.1f}%)")

In [ ]:
def iq_to_complex(sig):
    """Convert (N, 2) IQ signal to complex array."""
    return sig[:, 0] + 1j * sig[:, 1]

# Get sample indices for each class
def get_class_samples(labels, signals, n_samples=5):
    """Get sample indices for each class."""
    samples = {}
    for c in np.unique(labels):
        idx = np.where(labels == c)[0][:n_samples]
        samples[c] = signals[idx]
    return samples

train_samples = get_class_samples(train_labels, train_signals, n_samples=50)
test_samples = get_class_samples(test_labels, test_signals, n_samples=50)

---
## 1. Nonlinear Power Transforms (x², x⁴)

For BPSK (Class 6), x² maps {0, π} states to 0 rad → spectral line at 2× carrier.
For QPSK (Class 8), x⁴ collapses four quadrants → spectral line at 4× carrier.

In [ ]:
def compute_power_transform_features(sig):
    """
    Compute features from nonlinear power transforms.
    
    Returns:
        Dictionary with features from x², x⁴ transforms
    """
    x = iq_to_complex(sig)
    n = len(x)
    
    # Square transform: y = x²
    x2 = x ** 2
    spec_x2 = np.abs(fft(x2))[:n//2]
    spec_x2_norm = spec_x2 / (spec_x2.sum() + 1e-10)
    
    # Quad transform: y = x⁴
    x4 = x ** 4
    spec_x4 = np.abs(fft(x4))[:n//2]
    spec_x4_norm = spec_x4 / (spec_x4.sum() + 1e-10)
    
    features = {
        # x² spectrum features
        'x2_spec_max': spec_x2.max(),
        'x2_spec_peak_ratio': spec_x2.max() / (spec_x2.mean() + 1e-10),
        'x2_spec_kurtosis': kurtosis(spec_x2),
        'x2_spec_entropy': -np.sum(spec_x2_norm * np.log(spec_x2_norm + 1e-10)),
        
        # x⁴ spectrum features  
        'x4_spec_max': spec_x4.max(),
        'x4_spec_peak_ratio': spec_x4.max() / (spec_x4.mean() + 1e-10),
        'x4_spec_kurtosis': kurtosis(spec_x4),
        'x4_spec_entropy': -np.sum(spec_x4_norm * np.log(spec_x4_norm + 1e-10)),
        
        # Phase after transforms
        'x2_phase_std': np.std(np.angle(x2)),
        'x4_phase_std': np.std(np.angle(x4)),
        
        # Amplitude after transforms
        'x2_amp_std': np.std(np.abs(x2)),
        'x4_amp_std': np.std(np.abs(x4)),
    }
    
    return features

In [ ]:
# Visualize power transforms for each class
fig, axes = plt.subplots(3, 9, figsize=(18, 10))

all_classes = list(range(6)) + [6, 7, 8]
class_names = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'A6 (BPSK)', 'A7 (Const)', 'A8 (QPSK)']

for i, c in enumerate(all_classes):
    if c < 6:
        sig = train_samples[c][0]
    else:
        sig = test_samples[c][0]
    
    x = iq_to_complex(sig)
    n = len(x)
    freqs = fftfreq(n)[:n//2]
    
    # Original spectrum
    spec = np.abs(fft(x))[:n//2]
    axes[0, i].semilogy(freqs, spec, 'b-', lw=0.5)
    axes[0, i].set_title(class_names[i], fontsize=9)
    if i == 0:
        axes[0, i].set_ylabel('|FFT(x)|')
    
    # x² spectrum
    x2 = x ** 2
    spec_x2 = np.abs(fft(x2))[:n//2]
    axes[1, i].semilogy(freqs, spec_x2, 'r-', lw=0.5)
    if i == 0:
        axes[1, i].set_ylabel('|FFT(x²)|')
    
    # x⁴ spectrum
    x4 = x ** 4
    spec_x4 = np.abs(fft(x4))[:n//2]
    axes[2, i].semilogy(freqs, spec_x4, 'g-', lw=0.5)
    if i == 0:
        axes[2, i].set_ylabel('|FFT(x⁴)|')
    axes[2, i].set_xlabel('Freq')

plt.suptitle('Nonlinear Power Transforms: Spectral Analysis', fontsize=12)
plt.tight_layout()
plt.savefig('plots_power_transforms.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_power_transforms.png")

In [ ]:
# Compute power transform features for all signals
print("Computing power transform features...")

def extract_features_for_dataset(signals):
    features_list = []
    for sig in signals:
        features_list.append(compute_power_transform_features(sig))
    feature_names = list(features_list[0].keys())
    features = np.array([[f[n] for n in feature_names] for f in features_list])
    return features, feature_names

train_power_feats, power_feat_names = extract_features_for_dataset(train_signals)
test_power_feats, _ = extract_features_for_dataset(test_signals)

print(f"Features: {power_feat_names}")

In [ ]:
# Evaluate AUC for each power transform feature
print("\n=== Power Transform Features AUC ===")
power_aucs = {}
for i, name in enumerate(power_feat_names):
    try:
        auc = roc_auc_score(test_binary, test_power_feats[:, i])
        # Try inverted if AUC < 0.5
        if auc < 0.5:
            auc = 1 - auc
        power_aucs[name] = auc
        print(f"{name:25s}: AUC = {auc:.3f}")
    except:
        print(f"{name:25s}: ERROR")

best_power = max(power_aucs, key=power_aucs.get)
print(f"\nBest power transform feature: {best_power} (AUC={power_aucs[best_power]:.3f})")

---
## 2. Symbol-Lagged Correlation (τ ≈ 3.4)

Using the retro-engineered symbol rate, apply lagged conjugate product:
$$y(t) = x(t) \cdot x^*(t - \tau) \quad \text{where } \tau \approx 3.4$$

For Constant Envelope signals (Class 7), phase remains stable during symbol duration.

In [ ]:
def compute_symbol_lag_features(sig, tau_values=[3, 4, 5, 10, 20]):
    """
    Compute features from symbol-lagged correlation.
    
    y(t) = x(t) * conj(x(t-tau))
    """
    x = iq_to_complex(sig)
    features = {}
    
    for tau in tau_values:
        # Lagged conjugate product
        y = x[tau:] * np.conj(x[:-tau])
        
        # Phase of product = phase difference
        phase_diff = np.angle(y)
        
        # Features from this lag
        features[f'lag{tau}_phase_std'] = np.std(phase_diff)
        features[f'lag{tau}_phase_mean'] = np.abs(np.mean(phase_diff))
        features[f'lag{tau}_amp_std'] = np.std(np.abs(y))
        features[f'lag{tau}_amp_mean'] = np.mean(np.abs(y))
        
        # Phase stability: count "plateaus" where phase is stable
        phase_changes = np.abs(np.diff(phase_diff))
        stable_fraction = np.mean(phase_changes < 0.1)
        features[f'lag{tau}_stable_frac'] = stable_fraction
        
        # Kurtosis of phase changes (should be high for discrete modulations)
        features[f'lag{tau}_phase_change_kurtosis'] = kurtosis(phase_changes)
    
    return features

In [ ]:
# Visualize symbol-lagged correlation
fig, axes = plt.subplots(3, 9, figsize=(18, 10))

tau = 3  # Close to symbol rate of 3.4

for i, c in enumerate(all_classes):
    if c < 6:
        sig = train_samples[c][0]
    else:
        sig = test_samples[c][0]
    
    x = iq_to_complex(sig)
    y = x[tau:] * np.conj(x[:-tau])
    phase_diff = np.angle(y)
    
    # Time slice
    t_slice = slice(0, 500)
    
    # Phase difference over time
    axes[0, i].plot(phase_diff[t_slice], 'b-', lw=0.5)
    axes[0, i].set_title(class_names[i], fontsize=9)
    axes[0, i].set_ylim(-np.pi, np.pi)
    if i == 0:
        axes[0, i].set_ylabel(f'Phase diff (τ={tau})')
    
    # Histogram of phase differences
    axes[1, i].hist(phase_diff, bins=50, density=True, alpha=0.7)
    if i == 0:
        axes[1, i].set_ylabel('Phase diff dist')
    
    # IQ plot of y = x(t) * conj(x(t-tau))
    axes[2, i].scatter(y.real[::10], y.imag[::10], s=1, alpha=0.3)
    axes[2, i].set_aspect('equal')
    if i == 0:
        axes[2, i].set_ylabel('y = x·x*(t-τ)')

plt.suptitle(f'Symbol-Lagged Correlation Analysis (τ={tau})', fontsize=12)
plt.tight_layout()
plt.savefig('plots_symbol_lag.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_symbol_lag.png")

In [ ]:
# Compute symbol lag features
print("Computing symbol-lagged correlation features...")

def extract_lag_features(signals):
    features_list = []
    for sig in signals:
        features_list.append(compute_symbol_lag_features(sig))
    feature_names = list(features_list[0].keys())
    features = np.array([[f[n] for n in feature_names] for f in features_list])
    return features, feature_names

train_lag_feats, lag_feat_names = extract_lag_features(train_signals)
test_lag_feats, _ = extract_lag_features(test_signals)

# Evaluate AUCs
print("\n=== Symbol-Lag Features AUC ===")
lag_aucs = {}
for i, name in enumerate(lag_feat_names):
    try:
        auc = roc_auc_score(test_binary, test_lag_feats[:, i])
        if auc < 0.5:
            auc = 1 - auc
        lag_aucs[name] = auc
        print(f"{name:30s}: AUC = {auc:.3f}")
    except:
        pass

best_lag = max(lag_aucs, key=lag_aucs.get)
print(f"\nBest symbol-lag feature: {best_lag} (AUC={lag_aucs[best_lag]:.3f})")

---
## 3. Instantaneous Frequency Analysis (dφ/dt)

$$f_{inst} = \frac{1}{2\pi} \frac{d\phi}{dt}$$

FSK variants (Class 7?) show discrete, stable frequency levels ("staircase" patterns).
PSK classes show erratic spikes during transitions.

In [ ]:
def compute_inst_freq_features(sig):
    """
    Compute instantaneous frequency features.
    """
    x = iq_to_complex(sig)
    
    # Unwrap phase to avoid discontinuities
    phase = np.unwrap(np.angle(x))
    
    # Instantaneous frequency = derivative of phase
    inst_freq = np.diff(phase) / (2 * np.pi)
    
    # Features
    features = {
        'if_mean': np.mean(inst_freq),
        'if_std': np.std(inst_freq),
        'if_kurtosis': kurtosis(inst_freq),
        'if_skew': skew(inst_freq),
        'if_min': np.min(inst_freq),
        'if_max': np.max(inst_freq),
        'if_range': np.max(inst_freq) - np.min(inst_freq),
        'if_median': np.median(inst_freq),
        'if_iqr': np.percentile(inst_freq, 75) - np.percentile(inst_freq, 25),
    }
    
    # Detect "staircase" patterns (discrete frequency levels)
    # Quantize to detect levels
    n_levels = len(np.unique(np.round(inst_freq, 3)))
    features['if_n_levels'] = n_levels
    
    # Histogram-based entropy
    hist, _ = np.histogram(inst_freq, bins=50, density=True)
    hist = hist / (hist.sum() + 1e-10)
    features['if_entropy'] = -np.sum(hist * np.log(hist + 1e-10))
    
    # Frequency transitions (detect sharp changes)
    freq_diff = np.abs(np.diff(inst_freq))
    features['if_diff_std'] = np.std(freq_diff)
    features['if_diff_max'] = np.max(freq_diff)
    features['if_spike_count'] = np.sum(freq_diff > 0.1)  # Count spikes
    features['if_stable_frac'] = np.mean(freq_diff < 0.01)  # Fraction of stable regions
    
    return features

In [ ]:
# Visualize instantaneous frequency
fig, axes = plt.subplots(3, 9, figsize=(18, 10))

for i, c in enumerate(all_classes):
    if c < 6:
        sig = train_samples[c][0]
    else:
        sig = test_samples[c][0]
    
    x = iq_to_complex(sig)
    phase = np.unwrap(np.angle(x))
    inst_freq = np.diff(phase) / (2 * np.pi)
    
    t_slice = slice(0, 500)
    
    # Phase over time
    axes[0, i].plot(phase[t_slice], 'b-', lw=0.5)
    axes[0, i].set_title(class_names[i], fontsize=9)
    if i == 0:
        axes[0, i].set_ylabel('Unwrapped Phase')
    
    # Instantaneous frequency
    axes[1, i].plot(inst_freq[t_slice], 'r-', lw=0.5)
    axes[1, i].set_ylim(-0.3, 0.3)
    if i == 0:
        axes[1, i].set_ylabel('Inst. Freq')
    
    # Histogram of inst freq
    axes[2, i].hist(inst_freq, bins=50, density=True, alpha=0.7, range=(-0.3, 0.3))
    if i == 0:
        axes[2, i].set_ylabel('Inst. Freq Dist')

plt.suptitle('Instantaneous Frequency Analysis (dφ/dt)', fontsize=12)
plt.tight_layout()
plt.savefig('plots_inst_freq_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_inst_freq_analysis.png")

In [ ]:
# Compute instantaneous frequency features
print("Computing instantaneous frequency features...")

def extract_if_features(signals):
    features_list = []
    for sig in signals:
        features_list.append(compute_inst_freq_features(sig))
    feature_names = list(features_list[0].keys())
    features = np.array([[f[n] for n in feature_names] for f in features_list])
    return features, feature_names

train_if_feats, if_feat_names = extract_if_features(train_signals)
test_if_feats, _ = extract_if_features(test_signals)

# Evaluate AUCs
print("\n=== Instantaneous Frequency Features AUC ===")
if_aucs = {}
for i, name in enumerate(if_feat_names):
    try:
        vals = test_if_feats[:, i]
        vals = np.nan_to_num(vals, nan=0, posinf=1e6, neginf=-1e6)
        auc = roc_auc_score(test_binary, vals)
        if auc < 0.5:
            auc = 1 - auc
        if_aucs[name] = auc
        print(f"{name:25s}: AUC = {auc:.3f}")
    except Exception as e:
        print(f"{name:25s}: ERROR - {e}")

best_if = max(if_aucs, key=if_aucs.get)
print(f"\nBest instantaneous freq feature: {best_if} (AUC={if_aucs[best_if]:.3f})")

---
## 4. Cyclostationary Feature Extraction (SCF)

Radio signals exhibit cyclostationary properties - statistics vary periodically with symbol rate.
Compute spectrum of instantaneous power |x(t)|² and look for peaks at cyclic frequency α = 1/3.4.

In [ ]:
def compute_cyclo_features(sig, expected_symbol_rate=3.4):
    """
    Compute cyclostationary features.
    
    Look for periodic patterns in the signal statistics at the symbol rate.
    """
    x = iq_to_complex(sig)
    n = len(x)
    
    # Instantaneous power
    power = np.abs(x) ** 2
    
    # Spectrum of instantaneous power (reveals cyclic frequencies)
    power_spec = np.abs(fft(power - power.mean()))[:n//2]
    freqs = fftfreq(n)[:n//2]
    
    # Expected cyclic frequency
    alpha = 1.0 / expected_symbol_rate
    
    features = {}
    
    # Look for peak at expected cyclic frequency
    alpha_idx = int(alpha * n)
    window = 5
    
    if alpha_idx < len(power_spec) - window:
        features['cyclo_peak_at_alpha'] = power_spec[alpha_idx-window:alpha_idx+window].max()
        features['cyclo_peak_ratio'] = features['cyclo_peak_at_alpha'] / (power_spec.mean() + 1e-10)
    else:
        features['cyclo_peak_at_alpha'] = 0
        features['cyclo_peak_ratio'] = 0
    
    # Overall spectral features of power
    features['power_spec_max'] = power_spec.max()
    features['power_spec_std'] = power_spec.std()
    features['power_spec_kurtosis'] = kurtosis(power_spec)
    
    # Find dominant cyclic frequency
    peak_idx = np.argmax(power_spec[10:]) + 10  # Skip DC component
    features['dominant_cyclo_freq'] = freqs[peak_idx] if peak_idx < len(freqs) else 0
    
    # ACF of power (reveals symbol period)
    power_centered = power - power.mean()
    acf = np.correlate(power_centered, power_centered, mode='full')
    acf = acf[len(acf)//2:]
    acf = acf / (acf[0] + 1e-10)
    
    # Find first significant peak in ACF (symbol period)
    peaks, properties = signal.find_peaks(acf[5:100], height=0.05, distance=2)
    if len(peaks) > 0:
        features['acf_power_first_peak'] = peaks[0] + 5
        features['acf_power_first_peak_height'] = properties['peak_heights'][0]
    else:
        features['acf_power_first_peak'] = 0
        features['acf_power_first_peak_height'] = 0
    
    # Squared signal spectrum (for BPSK detection)
    x2 = x ** 2
    x2_spec = np.abs(fft(x2))[:n//2]
    features['x2_cyclo_peak'] = x2_spec.max()
    features['x2_cyclo_ratio'] = x2_spec.max() / (x2_spec.mean() + 1e-10)
    
    return features

In [ ]:
# Visualize cyclostationary features
fig, axes = plt.subplots(3, 9, figsize=(18, 10))

for i, c in enumerate(all_classes):
    if c < 6:
        sig = train_samples[c][0]
    else:
        sig = test_samples[c][0]
    
    x = iq_to_complex(sig)
    n = len(x)
    
    # Instantaneous power
    power = np.abs(x) ** 2
    power_centered = power - power.mean()
    
    # Spectrum of power
    power_spec = np.abs(fft(power_centered))[:n//2]
    freqs = fftfreq(n)[:n//2]
    
    # ACF of power
    acf = np.correlate(power_centered, power_centered, mode='full')
    acf = acf[len(acf)//2:]
    acf = acf / (acf[0] + 1e-10)
    
    # Power over time
    axes[0, i].plot(power[:200], 'b-', lw=0.5)
    axes[0, i].set_title(class_names[i], fontsize=9)
    if i == 0:
        axes[0, i].set_ylabel('|x(t)|²')
    
    # Spectrum of power
    axes[1, i].semilogy(freqs[:200], power_spec[:200], 'r-', lw=0.5)
    axes[1, i].axvline(1/3.4, color='g', linestyle='--', alpha=0.5, label='α=1/3.4')
    if i == 0:
        axes[1, i].set_ylabel('Power Spectrum')
    
    # ACF of power
    axes[2, i].plot(acf[:100], 'g-', lw=0.5)
    axes[2, i].axhline(0, color='k', linestyle='-', alpha=0.3)
    if i == 0:
        axes[2, i].set_ylabel('ACF(|x|²)')

plt.suptitle('Cyclostationary Analysis: Power Periodicity', fontsize=12)
plt.tight_layout()
plt.savefig('plots_cyclostationary.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_cyclostationary.png")

In [ ]:
# Compute cyclostationary features
print("Computing cyclostationary features...")

def extract_cyclo_features(signals):
    features_list = []
    for sig in signals:
        features_list.append(compute_cyclo_features(sig))
    feature_names = list(features_list[0].keys())
    features = np.array([[f[n] for n in feature_names] for f in features_list])
    return features, feature_names

train_cyclo_feats, cyclo_feat_names = extract_cyclo_features(train_signals)
test_cyclo_feats, _ = extract_cyclo_features(test_signals)

# Evaluate AUCs
print("\n=== Cyclostationary Features AUC ===")
cyclo_aucs = {}
for i, name in enumerate(cyclo_feat_names):
    try:
        vals = test_cyclo_feats[:, i]
        vals = np.nan_to_num(vals, nan=0, posinf=1e6, neginf=-1e6)
        auc = roc_auc_score(test_binary, vals)
        if auc < 0.5:
            auc = 1 - auc
        cyclo_aucs[name] = auc
        print(f"{name:30s}: AUC = {auc:.3f}")
    except Exception as e:
        print(f"{name:30s}: ERROR - {e}")

best_cyclo = max(cyclo_aucs, key=cyclo_aucs.get)
print(f"\nBest cyclostationary feature: {best_cyclo} (AUC={cyclo_aucs[best_cyclo]:.3f})")

---
## 5. Combined Analysis: All Transform Features

In [ ]:
# Combine all features
print("Combining all transform features...")

# Combine feature arrays
train_all_feats = np.hstack([train_power_feats, train_lag_feats, train_if_feats, train_cyclo_feats])
test_all_feats = np.hstack([test_power_feats, test_lag_feats, test_if_feats, test_cyclo_feats])

# Combine feature names
all_feat_names = power_feat_names + lag_feat_names + if_feat_names + cyclo_feat_names

print(f"Total features: {len(all_feat_names)}")
print(f"Train shape: {train_all_feats.shape}")
print(f"Test shape: {test_all_feats.shape}")

In [ ]:
# Combine all AUCs and rank
all_aucs = {**power_aucs, **lag_aucs, **if_aucs, **cyclo_aucs}

# Sort by AUC
sorted_aucs = sorted(all_aucs.items(), key=lambda x: x[1], reverse=True)

print("\n=== Top 20 Features by AUC ===")
print(f"{'Rank':<5} {'Feature':<35} {'AUC':<8} {'Category'}")
print("-" * 70)

for rank, (name, auc) in enumerate(sorted_aucs[:20], 1):
    if name in power_aucs:
        cat = 'Power Transform'
    elif name in lag_aucs:
        cat = 'Symbol Lag'
    elif name in if_aucs:
        cat = 'Inst. Freq'
    else:
        cat = 'Cyclostationary'
    print(f"{rank:<5} {name:<35} {auc:.3f}    {cat}")

In [ ]:
# Visualize top features
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

top_features = [name for name, _ in sorted_aucs[:8]]

for idx, feat_name in enumerate(top_features):
    feat_idx = all_feat_names.index(feat_name)
    
    # Get values for each class
    for c in range(9):
        mask = test_labels == c
        vals = test_all_feats[mask, feat_idx]
        color = 'blue' if c < 6 else 'red'
        label = f'C{c}' if c < 6 else f'A{c}'
        axes[idx].hist(vals, bins=30, alpha=0.3, label=label, color=color, density=True)
    
    axes[idx].set_title(f'{feat_name}\nAUC={all_aucs[feat_name]:.3f}', fontsize=9)
    axes[idx].legend(fontsize=6, ncol=3)

plt.suptitle('Top 8 Discriminative Features from Reverse Engineering Transforms', fontsize=12)
plt.tight_layout()
plt.savefig('plots_top_transform_features.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_top_transform_features.png")

---
## 6. Anomaly Detection with Combined Transform Features

In [ ]:
# Train Isolation Forest on combined features
print("Training Isolation Forest on transform features...")

# Clean up NaN/Inf
train_clean = np.nan_to_num(train_all_feats, nan=0, posinf=1e6, neginf=-1e6)
test_clean = np.nan_to_num(test_all_feats, nan=0, posinf=1e6, neginf=-1e6)

# Standardize
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_clean)
test_scaled = scaler.transform(test_clean)

# Train IF
clf = IsolationForest(n_estimators=200, contamination=0.1, random_state=42, n_jobs=-1)
clf.fit(train_scaled)

# Predict
scores = -clf.score_samples(test_scaled)  # Higher = more anomalous

# Evaluate
auc = roc_auc_score(test_binary, scores)
print(f"\nOverall AUC: {auc:.3f}")

# Per-class AUC
for c in [6, 7, 8]:
    mask = (test_labels == c) | (test_labels < 6)
    binary_c = (test_labels[mask] == c).astype(int)
    auc_c = roc_auc_score(binary_c, scores[mask])
    print(f"Class {c} AUC: {auc_c:.3f}")

In [ ]:
# Find best F1 threshold
precision, recall, thresholds = precision_recall_curve(test_binary, scores)
f1_scores = 2 * precision * recall / (precision + recall + 1e-10)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else thresholds[-1]
best_f1 = f1_scores[best_idx]

print(f"\nBest F1: {best_f1:.3f}")
print(f"At threshold: {best_threshold:.3f}")
print(f"Precision: {precision[best_idx]:.3f}")
print(f"Recall: {recall[best_idx]:.3f}")

# Per-class detection at best threshold
predictions = (scores > best_threshold).astype(int)
print("\nPer-class detection rates:")
for c in [6, 7, 8]:
    mask = test_labels == c
    detection_rate = predictions[mask].mean()
    print(f"  Anomaly {c}: {100*detection_rate:.1f}%")

In [ ]:
# Compare with C42 baseline from deep-deep dive
from feature_extraction import compute_cumulants

print("\n=== Comparison with C42 Baseline ===")

# Compute C42 for test
test_c42 = np.array([compute_cumulants(iq_to_complex(s))['C42'] for s in test_signals])

# C42 alone AUC
c42_auc = roc_auc_score(test_binary, test_c42)
if c42_auc < 0.5:
    c42_auc = 1 - c42_auc
    test_c42 = -test_c42

print(f"C42 alone AUC: {c42_auc:.3f}")
print(f"Transform features AUC: {auc:.3f}")

In [ ]:
# Combined: Transform features + C42 filter
print("\n=== Combined Approach: C42 Filter + Transform Features ===")

# C42 filter threshold
C42_THRESHOLD = -0.35

# Stage 1: C42 filter (definite normals)
sure_normal = test_c42 < C42_THRESHOLD
print(f"Stage 1 - C42 < {C42_THRESHOLD}:")
print(f"  Samples: {sure_normal.sum()}")
print(f"  Anomalies in this group: {test_binary[sure_normal].sum()}")

# Stage 2: IF on remaining
ambiguous = ~sure_normal
print(f"\nStage 2 - Ambiguous samples: {ambiguous.sum()}")

# Train on ambiguous training samples
train_c42 = np.array([compute_cumulants(iq_to_complex(s))['C42'] for s in train_signals])
train_ambiguous = train_c42 >= C42_THRESHOLD

scaler2 = StandardScaler()
train_amb_scaled = scaler2.fit_transform(train_clean[train_ambiguous])
test_amb_scaled = scaler2.transform(test_clean[ambiguous])

clf2 = IsolationForest(n_estimators=200, contamination=0.4, random_state=42, n_jobs=-1)
clf2.fit(train_amb_scaled)

# Combined scores
final_scores = np.zeros(len(test_signals))
final_scores[sure_normal] = -10  # Definitely normal
final_scores[ambiguous] = -clf2.score_samples(test_amb_scaled)

# Evaluate combined
combined_auc = roc_auc_score(test_binary, final_scores)
print(f"\nCombined AUC: {combined_auc:.3f}")

# Best F1
precision, recall, thresholds = precision_recall_curve(test_binary, final_scores)
f1_scores = 2 * precision * recall / (precision + recall + 1e-10)
best_idx = np.argmax(f1_scores)
best_f1 = f1_scores[best_idx]
print(f"Best F1: {best_f1:.3f}")
print(f"Precision: {precision[best_idx]:.3f}")
print(f"Recall: {recall[best_idx]:.3f}")

---
## 7. Per-Anomaly Class Analysis

In [ ]:
# Per-class feature comparison
print("=== Per-Anomaly Class Feature Analysis ===")

for anomaly_class in [6, 7, 8]:
    print(f"\n--- Anomaly Class {anomaly_class} ---")
    
    # Find features that best separate this anomaly from similar known classes
    if anomaly_class in [6, 8]:  # Similar to class 3 based on C42
        similar_classes = [3]
    else:  # Class 7 similar to 4, 5
        similar_classes = [4, 5]
    
    # Create binary: anomaly vs similar known
    mask_anomaly = test_labels == anomaly_class
    mask_similar = np.isin(test_labels, similar_classes)
    mask = mask_anomaly | mask_similar
    
    binary_local = test_labels[mask] == anomaly_class
    feats_local = test_all_feats[mask]
    
    # Rank features
    local_aucs = {}
    for i, name in enumerate(all_feat_names):
        try:
            vals = np.nan_to_num(feats_local[:, i], nan=0, posinf=1e6, neginf=-1e6)
            auc = roc_auc_score(binary_local, vals)
            if auc < 0.5:
                auc = 1 - auc
            local_aucs[name] = auc
        except:
            pass
    
    # Top 5 for this anomaly
    sorted_local = sorted(local_aucs.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"  Best features vs classes {similar_classes}:")
    for name, auc in sorted_local:
        print(f"    {name}: AUC={auc:.3f}")

In [ ]:
# Summary visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Feature category comparison
ax = axes[0, 0]
categories = ['Power\nTransform', 'Symbol\nLag', 'Inst.\nFreq', 'Cyclo-\nstationary']
cat_aucs = [
    max(power_aucs.values()) if power_aucs else 0,
    max(lag_aucs.values()) if lag_aucs else 0,
    max(if_aucs.values()) if if_aucs else 0,
    max(cyclo_aucs.values()) if cyclo_aucs else 0
]
colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6']
bars = ax.bar(categories, cat_aucs, color=colors)
ax.set_ylabel('Best AUC')
ax.set_title('Best AUC by Transform Category')
ax.set_ylim(0.5, 1.0)
for bar, auc in zip(bars, cat_aucs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{auc:.3f}', ha='center', fontsize=9)

# 2. ROC curve
ax = axes[0, 1]
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(test_binary, final_scores)
ax.plot(fpr, tpr, 'b-', lw=2, label=f'Combined (AUC={combined_auc:.3f})')
fpr2, tpr2, _ = roc_curve(test_binary, scores)
ax.plot(fpr2, tpr2, 'r--', lw=2, label=f'Transform only (AUC={auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves')
ax.legend()

# 3. Score distributions
ax = axes[1, 0]
ax.hist(final_scores[test_binary == 0], bins=50, alpha=0.5, label='Known (0-5)', density=True)
ax.hist(final_scores[test_binary == 1], bins=50, alpha=0.5, label='Anomaly (6-8)', density=True)
ax.set_xlabel('Anomaly Score')
ax.set_ylabel('Density')
ax.set_title('Score Distributions')
ax.legend()

# 4. Per-class detection
ax = axes[1, 1]
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else thresholds[-1]
predictions = (final_scores > best_threshold).astype(int)

class_rates = []
class_labels = []
for c in range(9):
    mask = test_labels == c
    if c < 6:
        # False positive rate for known classes
        rate = predictions[mask].mean()
        class_labels.append(f'C{c}\n(FP)')
    else:
        # True positive rate for anomalies
        rate = predictions[mask].mean()
        class_labels.append(f'A{c}\n(TP)')
    class_rates.append(rate)

colors = ['blue']*6 + ['red']*3
ax.bar(class_labels, class_rates, color=colors, alpha=0.7)
ax.set_ylabel('Detection Rate')
ax.set_title('Per-Class Detection Rates at Best F1 Threshold')
ax.axhline(0.5, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('plots_transform_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: plots_transform_summary.png")

---
## 8. Conclusions

In [ ]:
print("="*70)
print("REVERSE ENGINEERING TRANSFORMS - SUMMARY")
print("="*70)

print("\n1. POWER TRANSFORMS (x², x⁴)")
print(f"   Best feature: {best_power} (AUC={power_aucs[best_power]:.3f})")
print("   Insight: x² and x⁴ transforms reveal modulation structure,")
print("   but didn't significantly outperform simpler features.")

print("\n2. SYMBOL-LAGGED CORRELATION (τ ≈ 3.4)")
print(f"   Best feature: {best_lag} (AUC={lag_aucs[best_lag]:.3f})")
print("   Insight: Phase stability between symbols varies by modulation,")
print("   provides complementary discrimination.")

print("\n3. INSTANTANEOUS FREQUENCY (dφ/dt)")
print(f"   Best feature: {best_if} (AUC={if_aucs[best_if]:.3f})")
print("   Insight: IF statistics help distinguish modulation types,")
print("   if_std is particularly useful.")

print("\n4. CYCLOSTATIONARY FEATURES")
print(f"   Best feature: {best_cyclo} (AUC={cyclo_aucs[best_cyclo]:.3f})")
print("   Insight: Power spectrum features reveal cyclic structure,")
print("   useful for regime separation.")

print("\n" + "="*70)
print("OVERALL RESULTS")
print("="*70)
print(f"\nTransform features alone: AUC={auc:.3f}, F1={best_f1:.3f}")
print(f"Combined (C42 filter + transforms): AUC={combined_auc:.3f}")
print(f"\nTop 5 features overall:")
for i, (name, auc_val) in enumerate(sorted_aucs[:5], 1):
    print(f"  {i}. {name}: AUC={auc_val:.3f}")